In [ ]:
# To study the interdependencies between DED and insomnia and uncover patterns that are common for certain groups

In [31]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

#Import some packages for visualization
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [32]:
# path to the dataset
file_path = "Dry_Eye_Dataset.csv"

# Load the latest version of the file
df0 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "dakshnagra/dry-eye-disease",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

/var/folders/h4/8_8xkn4579zbrdb79s7zrtl00000gp/T/ipykernel_29205/3521891120.py:5: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df0 = kagglehub.load_dataset(


In [33]:
df.columns

Index(['gender', 'age', 'sleep_duration', 'sleep_quality', 'stress_level',
       'heart_rate', 'daily_steps', 'physical_activity', 'height', 'weight',
       'sleep_disorder', 'wake_up_during_night', 'feel_sleepy_during_day',
       'caffeine_consumption', 'alcohol_consumption', 'smoking',
       'medical_issue', 'ongoing_medication', 'smart_device_before_bed',
       'average_screen_time', 'blue_light_filter', 'discomfort_eye_strain',
       'redness_in_eye', 'eye_irritation', 'dry_eye_disease', 'bp_category',
       'Insomnia', 'insomnia'],
      dtype='object')

In [34]:
df = df0.copy()

#Make everything lower case, and replace space with '_'
df.columns = [col.lower().replace(" ", "_").replace("-", "_").replace("itchiness/irritation_in_eye", 'eye_irritation') for col in df.columns]

# Split the blood_pressure column into two new columns
df[['systolic', 'diastolic']] = df['blood_pressure'].str.split('/', expand=True)

# Define the Blood Pressure Category
def categorize_bp(row):
    if 110 <= int(row['systolic']) <= 120 and 70 <= int(row['diastolic']) <= 80:
        return 'normal'
    elif int(row['systolic']) > 120 or int(row['diastolic']) > 80:
        return 'high'
    else:
        return 'low'
# New BP category column
df['bp_category'] = df.apply(categorize_bp, axis=1)

In [35]:
omit_cols = ['blood_pressure',
              'systolic',
              'diastolic']
df = df.drop(columns=omit_cols)

In [48]:
df.columns

Index(['gender', 'age', 'sleep_duration', 'sleep_quality', 'stress_level',
       'heart_rate', 'daily_steps', 'physical_activity', 'height', 'weight',
       'sleep_disorder', 'wake_up_during_night', 'feel_sleepy_during_day',
       'caffeine_consumption', 'alcohol_consumption', 'smoking',
       'medical_issue', 'ongoing_medication', 'smart_device_before_bed',
       'average_screen_time', 'blue_light_filter', 'discomfort_eye_strain',
       'redness_in_eye', 'eye_irritation', 'dry_eye_disease', 'bp_category',
       'insomnia'],
      dtype='object')

In [49]:
#Creating the target for insomnia from: 
#Sleep disorder, sleep quality, sleep duration, 'Feel sleepy during day', and 'Wake up during night'

In [53]:
def define_insomnia(row):
    criteria_met = 0
    
    # Check if each criterion is met
    if row['sleep_disorder'] == 'Y':  # Only consider those with sleep disorder
        criteria_met+=1
    if row['sleep_duration'] < 6:
            criteria_met += 1
    if row['sleep_quality'] <= 2:
        criteria_met += 1
    if row['feel_sleepy_during_day'] == 'Y':
        criteria_met += 1
    if row['wake_up_during_night'] == 'Y':
        criteria_met += 1
    
    # Assign insomnia as 1 if 2 or more criteria are met
    if criteria_met >= 3:
        return 1
    return 0  # Insomnia = 0 if fewer than 2 criteria are met

# Apply the function to create a new column 'Insomnia'
df['insomnia'] = df.apply(define_insomnia, axis=1)

In [62]:
'''
This shows why including sleep disorder as a general condition instead as a first filter is a better approach, since sleep disorder can
include conditions other than insomnia. For example, for this person, although sleep disorder is Y, the other conditions for insomnia
are not satisfied.
'''

print(df.iloc[1999])

gender                         M
age                           41
sleep_duration               8.2
sleep_quality                  4
stress_level                   2
heart_rate                    85
daily_steps                11000
physical_activity             48
height                       179
weight                        61
sleep_disorder                 Y
wake_up_during_night           N
feel_sleepy_during_day         N
caffeine_consumption           N
alcohol_consumption            N
smoking                        N
medical_issue                  N
ongoing_medication             N
smart_device_before_bed        Y
average_screen_time          2.2
blue_light_filter              Y
discomfort_eye_strain          N
redness_in_eye                 N
eye_irritation                 Y
dry_eye_disease                Y
bp_category                 high
insomnia                       0
Name: 1999, dtype: object


In [55]:
df['insomnia'].value_counts()

insomnia
0    12104
1     7896
Name: count, dtype: int64